# Rigorous Framework

Andrew David Burbanks 2018, 2019, 2020.

## Source files

Here are the source code files for the Python version of the rigorous framework.

1. [Manifest](#man)
2. [Interval arithmetic](#int)
3. [Truncated nonrigorous polynomials](#pol)
4. [Truncated rigorous series](#tru)
5. [Standard function balls](#bal)
6. [General function balls](#fun)
7. [Rectangle arithmetic](#rec)
8. [Coverings / coarse curves](#cov)
9. [Real interval matrices](#rea)
10. [Tests and results](#tes)

<a name="man" href="man" id="man"></a>
### Manifest

In [14]:
!ls rigorous/*.py

rigorous/ball.py      rigorous/rectangle.py	 rigorous/test_interval.py
rigorous/covering.py  rigorous/rmatrix.py	 rigorous/test_int.py
rigorous/function.py  rigorous/rounding.py	 rigorous/test_poly.py
rigorous/__init__.py  rigorous/test_ball.py	 rigorous/test_trunc.py
rigorous/interval.py  rigorous/test_float.py	 rigorous/trunc.py
rigorous/poly.py      rigorous/test_function.py


<a name="int" href="int" id="int"></a>
### Interval arithmetic

In [4]:
cat rigorous/interval.py

from decimal import Decimal, getcontext
from decimal import localcontext, ROUND_CEILING, ROUND_FLOOR, Inexact

RR = Decimal

str_prec = 15

def set_precision(prec):
    getcontext().prec = prec
    
def set_output_precision(prec):
    global str_prec
    str_prec = prec
    
def quantize(x):
    """
    
    Safely convert interval to a simpler one in which only the final dp differs.
    This is used to indicate which digits of the number are known for certain.
    
    """
    rho = RR(10)**(int(abs(x.hi-x.lo).log10()))
    a = x.lo.quantize(rho, rounding=ROUND_FLOOR)
    b = x.hi.quantize(rho, rounding=ROUND_CEILING)
    return Interval(a, b)

def add_up(a, b):
    with localcontext() as ctx:
        ctx.rounding = ROUND_CEILING
        return a+b

def add_dn(a, b):
    with localcontext() as ctx:
        ctx.rounding = ROUND_FLOOR
        return a+b

def mul_up(a, b):
    with localcontext() as ctx:
        ctx.rounding = ROUND_CEILING
        return a*b

def mul_dn(a, b):
    with 

<a name="rec" href="rec" id="rec"></a>
### Rectangle arithmetic

In [5]:
cat rigorous/rectangle.py

from .interval import Interval, RR

class Rectangle:

    def __init__(self, re, im=None):
        if isinstance(re, (RR, float, int)):
            re = Interval(re, re)
        if im is None:
            im = Interval(0, 0)
        if isinstance(im, (RR, float, int)):
            im = Interval(im, im)
        assert isinstance(re, Interval)
        assert isinstance(im, Interval)
        self.re = re
        self.im = im

    def __repr__(self):
        return f'Rectangle(re={self.re}, im={self.im})'

    # inspectors
    
    def real(self):
        return self.re

    def imag(self):
        return self.im

    # unary arithmetic operators

    def __pos__(self):
        re = self.re
        im = self.im
        return Rectangle(re, im)

    def __neg__(self):
        re = -self.re
        im = -self.im
        return Rectangle(re, im)

    # binary arithmetic operators

    def __add__(self, other):
        if isinstance(other, (Interval, RR, float, int)):
            other = Recta

<a name="pol" href="pol" id="pol"></a>
### Polynomials (nonrigorous) with truncated operations

In [6]:
cat rigorous/poly.py

def sumup(seq):
    elts = iter(seq)
    total = next(elts)
    for x in elts:
        total = total + x
    return total


from itertools import zip_longest, islice


def trim(coeffs):
    # this should be fairly fast
    # this works for intervals by using 0*coeffs[k] below
    for k in reversed(range(len(coeffs))):
        if not coeffs[k] == 0 * coeffs[k]:
            break
    return coeffs[:max(1, k + 1)]


class Poly:
    """
    We represent the zero polynomial by the degree zero Poly1d([0]).
    This ensures that at least one coefficient is present.
    This coefficient is used in various places to produce the relevant unit.

    The safest way to test if a polynomial is zero is whether self*0 == self.
    """

    def __init__(self, coeffs):
        assert len(coeffs) >= 1
        self.coeffs = list(trim(coeffs))

    @property
    def degree(self):
        return len(self.coeffs) - 1

    def __repr__(self):
        return 'Poly(%r)' % (self.coeffs)

    def __len__(self):
 

<a name="tru" href="tru" id="tru"></a>
### Series (rigorous) with truncated operations

In [7]:
cat rigorous/trunc.py

from itertools import islice
from .interval import Interval, RR


def sumup(seq):
    elts = iter(seq)
    total = next(elts)
    for x in elts:
        total = total + x
    return total


class Trunc:

    # this class could be improved in efficiency
    # we could allow shorter coefficient lists than the truncation degree
    # using zip would allow padding by coeff[0]*0 with zip_longest
    # this would need some amendments to multiplication, but the unit tests
    # would help to ensure that the code didn't break, plus we can keep
    # the slow version for comparison.

    def __init__(self, truncation_degree, coeffs):
        assert truncation_degree >= 0
        assert 1 <= len(coeffs)  # <= truncation_degree + 1
        self.truncation_degree = truncation_degree
        self.coeffs = list(coeffs[:min(len(coeffs), truncation_degree + 1)])
        for k in range(len(self.coeffs), truncation_degree + 1):
            zero = self.coeffs[0] * 0
            self.coeffs.append(zero)
 

<a name="bal" href="bal" id="bal"></a>
### Standard function balls (disc algebra; unit disc)

In [8]:
cat rigorous/ball.py

from .interval import RR, Interval, Pub
from .rectangle import Rectangle
from .poly import Poly
from .trunc import Trunc

def bound_quasi_power(k0, g_norm):
    assert g_norm < Interval(1.0, 1.0)
    k = k0 + 1
    latest = k * g_norm ** (k - 1)
    best = latest
    while latest >= best:
        best = latest
        k = k + 1
        latest = k * g_norm ** (k - 1)
    return best


class Ball:

    def __init__(self, P, H, G):
        assert isinstance(P, Trunc)
        assert all(isinstance(a, Interval) for a in P)
        self.P = P
        self.H = Pub(H) # Ball.__init__ ensures interval [0, hi]
        self.G = Pub(G) # Ball.__init__ ensures interval [0, hi]

    def __repr__(self):
        return 'Ball(P=%r, H=%r, G=%r)' % (self.P, self.H, self.G)

    @property
    def truncation_degree(self):
        return self.P.truncation_degree

    def __eq__(self, other):
        # for comparing balls, not the functions contained within them
        assert isinstance(other, Ball)
       

<a name="fun" href="fun" id="fun"></a>
### General function balls (disc algebra; general disc)

In [9]:
cat rigorous/function.py

from .interval import Interval, RR, str_prec
from .rectangle import Rectangle
from .ball import Ball
from .trunc import Trunc

ArgTypes = (Rectangle, Interval, RR, float, int)

class Domain:

    """
    A disc domain.  The centre and radius can be non-rigorous or rigorous.
    """

    def __init__(self, centre, radius):
        self.c = centre
        self.r = radius

    def to_unit(self, x):
        u = (x - self.c) * (1 / self.r)
        return u

    def from_unit(self, u):
        x = self.c + u * self.r
        return x

    def __eq__(self, other):
        assert isinstance(other, Domain)
        return self.c == other.c and self.r == other.r

    def __repr__(self):
        return f'Domain(centre={self.c}, radius={self.r})'


class Function:

    """
    A Function object represents a power series expanded on some domain.
    """

    def __init__(self, dom, func):
        self.dom = dom
        self.func = func

    def __eq__(self, other):
        assert isinstance(other, F

<a name="rea" href="rea" id="rea"></a>
### Real interval matrices

In [10]:
cat rigorous/rmatrix.py

from rigorous.interval import RR, Interval
from rigorous.trunc import Trunc
from rigorous.ball import Ball
from rigorous.function import Function

class RMatrix:

    """
    Matrices of intervals.
    """

    @staticmethod
    def promote(x):
        if isinstance(x, (int, float)):
            return Interval(RR(x))
        if isinstance(x, RR):
            return Interval(x)
        if isinstance(x, Interval):
            return x
        raise NotImplementedError

    @staticmethod
    def zeros(size):
        m, n = size
        a = [[Interval(0) for _ in range(n)] for _ in range(m)]
        return RMatrix(a)
    
    @staticmethod
    def eye(n):
        A = RMatrix.zeros((n, n))
        for k in range(n):
            A.elts[k][k] = Interval(1)
        return A
    
    @staticmethod
    def basis_element(n, k):
        a = [[Interval(0)] for j in range(n)]
        a[k][0] = Interval(1)
        return RMatrix(a)
        
    def __init__(self, a):
        self.elts = [[None for _

<a name="cov" href="cov" id="cov"></a>
### Rectangle coverings / "coarse curves"

In [11]:
cat rigorous/covering.py

from .interval import Interval, RR
from .rectangle import Rectangle

import math

def cover_interval(lo, hi, n):
    """
    Return a list of n overlapping subintervals covering the interval [lo, hi].
    """
    assert lo <= hi
    assert n >= 1
    params_vals = [Interval(k)/Interval(n) for k in range(n+1)]
    sample_vals = [p*Interval(hi) + (Interval(1)-p)*Interval(lo) for p in params_vals]
    x_vals = [Interval.hull([s0, s1]) for s0, s1 in zip(sample_vals[:-1], sample_vals[1:])]
    return x_vals

def cover_graph(f, lo, hi, n):
    """
    Return a list of n overlapping rectangles covering the graph of f on interval [lo, hi].
    """
    assert lo <= hi
    assert n >= 1
    x_vals = cover_interval(lo, hi, n)
    f_vals = [f(x) for x in x_vals]
    z_vals = [Rectangle(x, y) for x, y in zip(x_vals, f_vals)]
    return z_vals

def cover_circle(c, r, m, eps=2**-8):
    """
    Return a list of n=4m overlapping rectangles covering a circle centre c, radius r.
    The parameter eps co

<a name="tes" href="tes" id="tes"></a>
## Unit testing and results

### Results

In [3]:
cat rigorous/TEST_RESULTS.TXT

============================= test session starts ==============================
platform linux -- Python 3.7.4, pytest-5.4.1, py-1.8.1, pluggy-0.13.1
rootdir: /home/andrewburbanks/Documents/Working/Research/renormalisation/rigorous-py/___github___/rigorous
plugins: remotedata-0.3.2, doctestplus-0.5.0, html-1.20.0, arraydiff-0.3, metadata-1.8.0, openfiles-0.4.0, hypothesis-5.5.4, parallel-0.0.9, astropy-header-0.1.2
collected 1188 items
pytest-parallel: 7 workers (processes), 1 test per worker (thread)
...................................................................................................................................................................................................................................................................................................................................................................................... [  6%]
..................... [  6%]
...................................................................... [  6%]
...

### Test code

The test code is rather extensive.  There are some discussions of the relevant axioms in the code below.

In [13]:
cat rigorous/test_*.py

import pytest
import random

from .ball import Ball
from .poly import Poly
from .trunc import Trunc
from .interval import Interval, RR

from .test_interval import random_float_interval
from .test_poly import random_float_poly, random_int_poly, random_float_interval_poly
from .test_trunc import random_float_interval_trunc

# we intend to test using polys and zero bounds

num_test_cases = 10


def split_poly(p, n):
    assert len(p) >= 1
    p_lo = p.truncate(n)
    p_hi = p - p_lo
    return p_lo, p_hi


def random_float_interval_ball(n):
    p = random_float_interval_trunc(n)
    h = RR(random.random())
    g = RR(random.random())
    b = Ball(p, h, g)
    return b


def diff(seq):
    elts = iter(seq)
    a = next(elts)
    for b in elts:
        yield b - a
        a = b


def random_float_vector_bounded_norm(n, bound=1):
    # n elements needed
    # chosen as differences between n+1 sorted randoms on [0, 1)
    coeffs = list(diff(sorted(RR(random.random()) * bound for k in range(n 